In [2]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-ubuntu2204.pin
!sudo mv cuda-ubuntu2204.pin /etc/apt/preferences.d/cuda-repository-pin-600
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/3bf863cc.pub
!sudo add-apt-repository "deb https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/ /"
!sudo apt-get update
!sudo apt-get -y install cuda-toolkit-12-4

--2025-04-28 17:38:52--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-ubuntu2204.pin
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 2.16.106.132, 2.16.106.152
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|2.16.106.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [application/octet-stream]
Saving to: ‘cuda-ubuntu2204.pin’

cuda-ubuntu2204.pin 100%[===================>]     190  --.-KB/s    in 0s      

2025-04-28 17:38:52 (112 MB/s) - ‘cuda-ubuntu2204.pin’ saved [190/190]

Executing: /tmp/apt-key-gpghome.AhdyfcicPa/gpg.1.sh --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/3bf863cc.pub
gpg: requesting key from 'https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/3bf863cc.pub'
gpg: key A4B469963BF863CC: public key "cudatools <cudatools@nvidia.com>" imported
gpg: Total number processed: 1
gpg:            

In [3]:
import os
os.environ['PATH'] = '/usr/local/cuda-12.4/bin:' + os.environ['PATH']
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-12.4/lib64:' + os.environ['LD_LIBRARY_PATH']

In [6]:
# 4. Write your CUDA code to a file
%%writefile mat_multi.cu
#include <stdio.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <stdlib.h>
#include <iostream>

__global__ void matmul(int* A, int* B, long long int* C, int N) {
    int Row = blockIdx.y * blockDim.y + threadIdx.y;
    int Col = blockIdx.x * blockDim.x + threadIdx.x;
    if (Row < N && Col < N) {
        long long int Pvalue = 0;
        for (int k = 0; k < N; k++) {
            Pvalue += (long long int)A[Row * N + k] * B[k * N + Col];
        }
        C[Row * N + Col] = Pvalue;
    }
}

int main() {
     // Use a smaller matrix size for easier verification
    int N = 8;
    int size = N * N * sizeof(int);
    size_t result_size = N * N * sizeof(long long int);

    // Allocate CPU memory
    int* A = (int*)malloc(size);
    int* B = (int*)malloc(size);
    long long int* C = (long long int*)malloc(result_size);

    // Initialize matrices A and B with simple values
    // Matrix A: simple incremental values
    int A_data[64] = {
        1, 2, 3, 4, 5, 6, 7, 8,
        9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24,
        25, 26, 27, 28, 29, 30, 31, 32,
        33, 34, 35, 36, 37, 38, 39, 40,
        41, 42, 43, 44, 45, 46, 47, 48,
        49, 50, 51, 52, 53, 54, 55, 56,
        57, 58, 59, 60, 61, 62, 63, 64
    };

    // Matrix B: identity matrix for simplicity
    int B_data[64] = {
        1, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 1
    };

    // Copy the data to A and B
    memcpy(A, A_data, size);
    memcpy(B, B_data, size);

    // Print matrix A
    std::cout << "Matrix A:" << std::endl;
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            std::cout << A[i * N + j] << " ";
        }
        std::cout << std::endl;
    }

    // Print matrix B
    std::cout << "\nMatrix B:" << std::endl;
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            std::cout << B[i * N + j] << " ";
        }
        std::cout << std::endl;
    }

    // Allocate GPU memory
    int* dev_A, * dev_B;
    long long int* dev_C;
    cudaMalloc(&dev_A, size);
    cudaMalloc(&dev_B, size);
    cudaMalloc(&dev_C, result_size);

    // Copy A and B to device memory
    cudaMemcpy(dev_A, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_B, B, size, cudaMemcpyHostToDevice);

    // Define block and grid sizes
    dim3 dimBlock(8, 8); // 8×8 threads per block for the 8×8 matrix
    dim3 dimGrid(1, 1);  // Only need one block for 8×8 matrix

    // Timing GPU kernel execution
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

       // Launch kernel - THIS IS THE FIXED LINE:
    matmul<<<dimGrid, dimBlock>>>(dev_A, dev_B, dev_C, N);

    // Check for kernel launch errors
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        std::cout << "CUDA kernel launch error: " << cudaGetErrorString(err) << std::endl;
        cudaFree(dev_A);
        cudaFree(dev_B);
        cudaFree(dev_C);
        free(A);
        free(B);
        free(C);
        return -1;
    }

    // Add explicit synchronization
    cudaDeviceSynchronize();

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);

    // Copy result back to CPU
    cudaMemcpy(C, dev_C, result_size, cudaMemcpyDeviceToHost);

    // Print the resulting matrix C
    std::cout << "\nMatrix C (Result):" << std::endl;
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            std::cout << C[i * N + j] << " ";
        }
        std::cout << std::endl;
    }

    // Print execution time
    std::cout << "\nMatrix multiplication completed in " << milliseconds << " ms\n";

    // Verify CPU calculation for comparison
    std::cout << "\nCPU Verification:" << std::endl;
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            long long int value = 0;
            for (int k = 0; k < N; k++) {
                value += (long long int)A[i * N + k] * B[k * N + j];
            }
            std::cout << value << " ";
        }
        std::cout << std::endl;
    }

    // Free memory
    cudaFree(dev_A);
    cudaFree(dev_B);
    cudaFree(dev_C);
    free(A);
    free(B);
    free(C);

    return 0;
}


Overwriting mat_multi.cu


In [7]:
!nvcc -arch=sm_75 mat_multi.cu -o mat_multi.out

In [8]:
!./mat_multi.out

Matrix A:
1 2 3 4 5 6 7 8 
9 10 11 12 13 14 15 16 
17 18 19 20 21 22 23 24 
25 26 27 28 29 30 31 32 
33 34 35 36 37 38 39 40 
41 42 43 44 45 46 47 48 
49 50 51 52 53 54 55 56 
57 58 59 60 61 62 63 64 

Matrix B:
1 0 0 0 0 0 0 0 
0 1 0 0 0 0 0 0 
0 0 1 0 0 0 0 0 
0 0 0 1 0 0 0 0 
0 0 0 1 1 0 0 0 
0 0 0 0 0 1 0 0 
0 0 0 0 0 0 1 0 
0 0 0 0 0 0 0 1 

Matrix C (Result):
1 2 3 9 5 6 7 8 
9 10 11 25 13 14 15 16 
17 18 19 41 21 22 23 24 
25 26 27 57 29 30 31 32 
33 34 35 73 37 38 39 40 
41 42 43 89 45 46 47 48 
49 50 51 105 53 54 55 56 
57 58 59 121 61 62 63 64 

Matrix multiplication completed in 0.126176 ms

CPU Verification:
1 2 3 9 5 6 7 8 
9 10 11 25 13 14 15 16 
17 18 19 41 21 22 23 24 
25 26 27 57 29 30 31 32 
33 34 35 73 37 38 39 40 
41 42 43 89 45 46 47 48 
49 50 51 105 53 54 55 56 
57 58 59 121 61 62 63 64 
